# Fine-tunning Invis-hire-v1

* accelerate is a distributed training library for PyTorch by HuggingFace. It allows you to train your models on multiple GPUs or CPUs in parallel (distributed configurations), which can significantly speed up training in presence of multiple GPUs (we won't use it in our example).
* peft is a Python library by HuggingFace for efficient adaptation of pre-trained language models (PLMs) to various downstream applications without fine-tuning all the model's parameters. PEFT methods only fine-tune a small number of (extra) model parameters, thereby greatly decreasing the computational and storage costs.
* bitsandbytes by Tim Dettmers, is a lightweight wrapper around CUDA custom functions, in particular 8-bit optimizers, matrix multiplication (LLM.int8()), and quantization functions. It allows to run models stored in 4-bit precision: while 4-bit bitsandbytes stores weights in 4-bits, the computation still happens in 16 or 32-bit and here any combination can be chosen (float16, bfloat16, float32, and so on).
* transformers is a Python library for natural language processing (NLP). It provides a number of pre-trained models for NLP tasks such as text classification, question answering, and machine translation.
* trl is a full stack library by HuggingFace providing a set of tools to train transformer language models with Reinforcement Learning, from the Supervised Fine-tuning step (SFT), Reward Modeling step (RM) to the Proximal Policy Optimization (PPO) step.

In [24]:
!nvidia-smi

Wed May 29 22:10:28 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P0             58W /  300W |   16127MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Installations and imports

In [1]:
#!pip install -q -U torch --index-url https://download.pytorch.org/whl/cu117
!pip install -q -U "torch==2.1.2"
!pip install -q -U transformers=="4.40.0"
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl
!pip install -q -U peft
!pip install -q -U tensorboard

!pip install -q -U -i https://pypi.org/simple/ bitsandbytes

# Ensure GPUtil is installed
!pip install gputil

In [32]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3,4,5,6,7"
os.environ["TOKENIZERS_PARALLELISM"] = "true"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:50'

In [33]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split

print(f"pytorch version {torch.__version__}")

pytorch version 2.1.2+cu121


Disabling two features in PyTorch related to memory efficiency and speed during operations on the Graphics Processing Unit (GPU) specifically for the scaled dot product attention (SDPA) function.

In [34]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

## Data

In [35]:
df = pd.read_csv("/content/merged_raw_data.csv")

melted_df = pd.melt(df, id_vars=["Name", "Greenhouse \nInterview\nDecision"],
                    value_vars=["Part 2 - Turn 1", "Part 2 - Turn 2", "Part 2 - Turn 3", "Part 2 - Turn 4"],
                    var_name="Turn", value_name="Response")
df = melted_df.rename(columns={'Greenhouse \nInterview\nDecision': 'Decision'})

# Define a function to split the conversation into user and AI parts
def split_conversation(text):
    parts = text.split(" AI: ")
    user_part = parts[0].replace("User: ", "")  # Remove the 'User:' prefix
    ai_part = parts[1] if len(parts) > 1 else ''  # Handle cases where there might not be an AI response
    return pd.Series([user_part, ai_part])

# Apply the function to the conversation column
df[['Question', 'Answer']] = df['Response'].apply(split_conversation)

df = df[df['Answer'].notna() & (df['Answer'] != '')]

mapping = {'Yes': 'selected','Strong Yes': 'selected', 'Strong No': 'rejected', 'No': 'rejected'}
df['Decision'] = df['Decision'].replace(mapping)

print(df.value_counts('Decision'))

X_train = list()
X_test = list()
for sentiment in ["selected", "rejected"]:
    train, test  = train_test_split(df[df['Decision']==sentiment],
                                    train_size=600,
                                    test_size=200,
                                    random_state=42)
    X_train.append(train)
    X_test.append(test)

"""train, test  = train_test_split(df, test_size=0.2, random_state=42)
X_train.append(train)
X_test.append(test)"""

X_train = pd.concat(X_train).sample(frac=1, random_state=10)
X_test = pd.concat(X_test)

eval_idx = [idx for idx in df.index if idx not in list(X_train.index) + list(X_test.index)]
X_eval = df[df.index.isin(eval_idx)]
X_eval = (X_eval
          .groupby('Decision', group_keys=False)
          .apply(lambda x: x.sample(n=50, random_state=10, replace=True)))
X_train = X_train.reset_index(drop=True)

def generate_prompt(data_point):
    return f"""
      Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"

      [ QUESTION: {data_point["Question"]} ANSWER: {data_point["Answer"]}] = {data_point["Decision"]} """.strip()

def generate_test_prompt(data_point):
    return f"""
      Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"

     [ QUESTION: {data_point["Question"]} ANSWER: {data_point["Answer"]}] = """.strip()


X_train = pd.DataFrame(X_train.apply(generate_prompt, axis=1),
                       columns=["Response"])
X_eval = pd.DataFrame(X_eval.apply(generate_prompt, axis=1),
                      columns=["Response"])

y_true = X_test.Decision
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["Response"])

train_data = Dataset.from_pandas(X_train)
eval_data = Dataset.from_pandas(X_eval)


Decision
rejected    2306
selected     859
Name: count, dtype: int64


#useful functions

In [36]:
def evaluate(y_true, y_pred):
    labels = ['selected', 'rejected']
    mapping = {'selected': 1,'rejected': 0}
    def map_func(x):
        return mapping.get(x, 1)

    y_true = np.vectorize(map_func)(y_true)
    y_pred = np.vectorize(map_func)(y_pred)

    # Calculate accuracy
    accuracy = accuracy_score(y_true=y_true, y_pred=y_pred)
    print(f'Accuracy: {accuracy:.3f}')

    # Generate accuracy report
    unique_labels = set(y_true)  # Get unique labels

    for label in unique_labels:
        label_indices = [i for i in range(len(y_true))
                         if y_true[i] == label]
        label_y_true = [y_true[i] for i in label_indices]
        label_y_pred = [y_pred[i] for i in label_indices]
        accuracy = accuracy_score(label_y_true, label_y_pred)
        print(f'Accuracy for label {label}: {accuracy:.3f}')

    # Generate classification report
    class_report = classification_report(y_true=y_true, y_pred=y_pred)
    print('\nClassification Report:')
    print(class_report)

    # Generate confusion matrix
    conf_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[0, 1])
    print('\nConfusion Matrix:')
    print(conf_matrix)

In [37]:
def predict(test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(X_test))):
        prompt = X_test.iloc[i]["Response"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 1,
                        temperature = 0.0,
                       )
        result = pipe(prompt)
        print(result)
        answer = result[0]['generated_text'].split("=")[-1].strip()
        print(answer)

        if "rejected" in answer or "RE" in answer:
            print("rejected")
            y_pred.append("rejected")
        elif "selected" in answer or "SELECT" in answer:
            y_pred.append("selected")
            print("selected")
        else:
            print("selected-l")
            y_pred.append("selected")
    return y_pred

In [ ]:
import GPUtil
from datetime import datetime

def gpu_usage():
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        print(f"GPU: {gpu.name}, Memory Used: {gpu.memoryUsed}MB, Total Memory: {gpu.memoryTotal}MB, Memory Utilization: {gpu.memoryUtil*100}%")

def monitor_gpu(fn):
    def wrapper(*args, **kwargs):
        print("GPU usage before running the function:")
        gpu_usage()
        result = fn(*args, **kwargs)
        print("GPU usage after running the function:")
        gpu_usage()
        return result
    return wrapper

#Loading model

Next we need to take care of the model, which is a 8b-chat-hf (8 billion parameters, no RLHF, in the HuggingFace compatible format), loading from Kaggle models and quantization.

Model loading and quantization:

* First the code loads the Llama-3 language model from the Kaggle Models.
* Then the code gets the float16 data type from the torch library. This is the data type that will be used for the computations.
* Next, it creates a BitsAndBytesConfig object with the following settings:
    1. load_in_4bit: Load the model weights in 4-bit format.
    2. bnb_4bit_quant_type: Use the "nf4" quantization type. 4-bit NormalFloat (NF4), is a new data type that is information theoretically optimal for normally distributed weights.
    3. bnb_4bit_compute_dtype: Use the float16 data type for computations.
    4. bnb_4bit_use_double_quant: Do not use double quantization (reduces the average memory footprint by quantizing also the quantization constants and saves an additional 0.4 bits per parameter.).
* Then the code creates a AutoModelForCausalLM object from the pre-trained Llama-2 language model, using the BitsAndBytesConfig object for quantization.
* After that, the code disables caching for the model.
* Finally the code sets the pre-training token probability to 1.

Tokenizer loading:

* First, the code loads the tokenizer for the Llama-3 language model.
* Then it sets the padding token to be the end-of-sequence (EOS) token.
* Finally, the code sets the padding side to be "right", which means that the input sequences will be padded on the right side. This is crucial for correct padding direction (this is the way with Llama 2).

In [12]:
X_train.head()

,Response
0,Analyze the the of Question and answer enclose...
1,Analyze the the of Question and answer enclose...
2,Analyze the the of Question and answer enclose...
3,Analyze the the of Question and answer enclose...
4,Analyze the the of Question and answer enclose...


In [ ]:
!gsutil -m cp -r gs://vertex-model-garden-public-us/mistralai/Mixtral-8x7B-v0.1 /content/

Copying gs://vertex-model-garden-public-us/mistralai/Mixtral-8x7B-v0.1/.gitattributes...
Copying gs://vertex-model-garden-public-us/mistralai/Mixtral-8x7B-v0.1/README.md...
Copying gs://vertex-model-garden-public-us/mistralai/Mixtral-8x7B-v0.1/config.json...
Copying gs://vertex-model-garden-public-us/mistralai/Mixtral-8x7B-v0.1/generation_config.json...
Copying gs://vertex-model-garden-public-us/mistralai/Mixtral-8x7B-v0.1/model-00001-of-00019.safetensors...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://vertex-model-garden-public-us/mistralai/Mixtral-8x7B-v0.1/model-00002-of-00019.safetensors...
Copying gs://vertex-model-garden-public-us/mistralai/Mixtral-8x7B-v0.1/model-00003-of-00019.safetensors...
Copying gs://vertex-model-garden-public-us/mistralai/Mixtral-8x7B-v0.1/model-0

In [ ]:
!gsutil -m cp -r gs://llm-finetune/peft/base_model/llama3/llama3-8b-chat-hf /content/

Copying gs://llm-finetune/peft/base_model/llama3/llama3-8b-chat-hf/LICENSE...
Copying gs://llm-finetune/peft/base_model/llama3/llama3-8b-chat-hf/README.md...
Copying gs://llm-finetune/peft/base_model/llama3/llama3-8b-chat-hf/USE_POLICY.md...
Copying gs://llm-finetune/peft/base_model/llama3/llama3-8b-chat-hf/config.json...
Copying gs://llm-finetune/peft/base_model/llama3/llama3-8b-chat-hf/generation_config.json...
Copying gs://llm-finetune/peft/base_model/llama3/llama3-8b-chat-hf/model-00001-of-00004.safetensors...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://llm-finetune/peft/base_model/llama3/llama3-8b-chat-hf/model-00002-of-00004.safetensors...
Copying gs://llm-finetune/peft/base_model/llama3/llama3-8b-chat-hf/model-00003-of-00004.safetensors...
Copying gs://llm-finetune/pef

In [38]:
#model_name = "/content/Mixtral-8x7B-v0.1"

model_name = "/content/merged_model"

compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config,
    cache_dir='',
    use_cache = False,
    device_map = "auto",
)

model.config.use_cache = False
model.config.pretraining_tp = 1

max_seq_length = 2048
tokenizer = AutoTokenizer.from_pretrained(model_name, max_seq_length=max_seq_length)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
!nvidia-smi

Wed May 29 21:24:12 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P0             59W /  300W |     309MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [39]:
y_pred = predict(test, model, tokenizer)

evaluate(y_true, y_pred)

  0%|          | 2/400 [00:00<01:01,  6.51it/s]

[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: It is truly a beautiful story. What do you think is the overall purpose of this story? ANSWER: The overall purpose of the story about La Llorona is to convey a message about the significance of maternal love, the impact of loss, and the emotional bond that exists between a mother and a child. This love in this story transcends even the worst tragedy, so much so that it causes La Llorona to continue mourning the passing of her children even in the afterlife.] = selected'}]
selected
selected
[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: How do you think Rosa\'s courage changed the way the villagers saw La Llorona? Does it show us that when we try to understand and f

  1%|          | 3/400 [00:00<00:56,  6.98it/s]

[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: user: Do ghosts exist ?     ANSWER: There has not been any concrete evidence to prove that ghosts exist or not, but based on this story there is a strong indication that people in this village did experience some unnatural events, and perhaps that is all the evidence we need.] = rejected'}]
rejected
rejected


  1%|▏         | 5/400 [00:00<01:04,  6.12it/s]

...

[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: What does the disappearing of La Llorona into the mist at the end signify? ANSWER: La Llorona disappearing into the mist is like her spirit finally finding peace. By meeting Rosa and seeing her promise to love her family always, La Llorona changes and goes away, maybe because she finally found what she had been looking for for so long.] = rejected'}]
rejected
rejected
[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: What country does the La Llorona myth come from? ANSWER: La Llorona stems from Mexico and other Latin American countries] = rejected'}]
rejected
rejected


100%|█████████▉| 399/400 [00:51<00:00,  7.49it/s]

[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION:  Can you tell me about the horror film of La Llorona?  ANSWER:  Sure!  There are multiple films telling the story of La Lorona including three main ones, then a number of low-budget movies and other movies loosely based off the story.  The first of the main films was the 1933â€™s La Llorona which was filmed in Mexico.  Next came the 1960â€™s film by Rene Cardona also titled La Llorona and filmed in Mexico.  Lastly, the third film came out in 1963 and was titled The Curse of the Crying Woman which was directed by Rafael Baledon. I hope this helps!] = selected'}]
selected
selected
[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: Why would that be a reasonable predictio

100%|██████████| 400/400 [00:52<00:00,  7.69it/s]

[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: "I am afraid of you. Have you come here to hurt me?" whispered Rosa. ANSWER: "No, little one. I am here to find my lost children and to let others know to always value the ones that they love."] = rejected'}]
rejected
rejected
Accuracy: 0.570
Accuracy for label 0: 0.575
Accuracy for label 1: 0.565

Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.57      0.57       200
           1       0.57      0.56      0.57       200

    accuracy                           0.57       400
   macro avg       0.57      0.57      0.57       400
weighted avg       0.57      0.57      0.57       400


Confusion Matrix:
[[115  85]
 [ 87 113]]


# Fine Tunning

In the next cell we set everything ready for the fine-tuning. We configures and initializes a Simple Fine-tuning Trainer (SFTTrainer) for training a large language model using the Parameter-Efficient Fine-Tuning (PEFT) method, which should save time as it operates on a reduced number of parameters compared to the model's overall size. The PEFT method focuses on refining a limited set of (additional) model parameters, while keeping the majority of the pre-trained LLM parameters fixed. This significantly reduces both computational and storage expenses. Additionally, this strategy addresses the challenge of catastrophic forgetting, which often occurs during the complete fine-tuning of LLMs.

PEFTConfig:

The peft_config object specifies the parameters for PEFT. The following are some of the most important parameters:

* lora_alpha: The learning rate for the LoRA update matrices.
* lora_dropout: The dropout probability for the LoRA update matrices.
* r: The rank of the LoRA update matrices.
* bias: The type of bias to use. The possible values are none, additive, and learned.
* task_type: The type of task that the model is being trained for. The possible values are CAUSAL_LM and MASKED_LM.

TrainingArguments:

The training_arguments object specifies the parameters for training the model. The following are some of the most important parameters:

* output_dir: The directory where the training logs and checkpoints will be saved.
* num_train_epochs: The number of epochs to train the model for.
* per_device_train_batch_size: The number of samples in each batch on each device.
* gradient_accumulation_steps: The number of batches to accumulate gradients before updating the model parameters.
* optim: The optimizer to use for training the model.
* save_steps: The number of steps after which to save a checkpoint.
* logging_steps: The number of steps after which to log the training metrics.
* learning_rate: The learning rate for the optimizer.
* weight_decay: The weight decay parameter for the optimizer.
* fp16: Whether to use 16-bit floating-point precision.
* bf16: Whether to use BFloat16 precision.
* max_grad_norm: The maximum gradient norm.
* max_steps: The maximum number of steps to train the model for.
* warmup_ratio: The proportion of the training steps to use for warming up the learning rate.
* group_by_length: Whether to group the training samples by length.
* lr_scheduler_type: The type of learning rate scheduler to use.
* report_to: The tools to report the training metrics to.
* evaluation_strategy: The strategy for evaluating the model during training.

SFTTrainer:

The SFTTrainer is a custom trainer class from the TRL library. It is used to train large language models (also using the PEFT method).

The SFTTrainer object is initialized with the following arguments:

* model: The model to be trained.
* train_dataset: The training dataset.
* eval_dataset: The evaluation dataset.
* peft_config: The PEFT configuration.
* dataset_text_field: The name of the text field in the dataset.
* tokenizer: The tokenizer to use.
* args: The training arguments.
* packing: Whether to pack the training samples.
* max_seq_length: The maximum sequence length.

Once the SFTTrainer object is initialized, it can be used to train the model by calling the train() method

In [16]:
!nvidia-smi

Wed May 29 22:26:16 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P0             59W /  300W |     309MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [15]:
!rm -rf /content/trained_weigths

In [17]:
!ls

llama3-8b-chat-hf  merged_raw_data.csv


In [18]:
output_dir="trained_weigths"

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj",],
)

training_arguments = TrainingArguments(
    output_dir=output_dir,                    # directory to save and repository id
    num_train_epochs=3,                       # number of training epochs
    per_device_train_batch_size=1,            # batch size per device during training
    gradient_accumulation_steps=8,            # number of steps before performing a backward/update pass
    gradient_checkpointing=False,             # use gradient checkpointing to save memory
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,                         # log every 10 steps
    learning_rate=2e-4,                       # learning rate, based on QLoRA paper
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,                        # max gradient norm based on QLoRA paper
    max_steps=-1,
    warmup_ratio=0.03,                        # warmup ratio based on QLoRA paper
    group_by_length=False,
    lr_scheduler_type="cosine",               # use cosine learning rate scheduler
    report_to="tensorboard",                  # report metrics to tensorboard
    evaluation_strategy="epoch"               # save checkpoint every epoch
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_data,
    eval_dataset=eval_data,
    peft_config=peft_config,
    dataset_text_field="Response",
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    }
)

trainer.train()

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,1.262400,1.347556
2,1.101800,1.316454
3,0.909300,1.370996


TrainOutput(global_step=450, training_loss=1.1280824322170682, metrics={'train_runtime': 1563.8297, 'train_samples_per_second': 2.302, 'train_steps_per_second': 0.288, 'total_flos': 2.072802067255296e+16, 'train_loss': 1.1280824322170682, 'epoch': 3.0})

In [22]:
!nvidia-smi

Wed May 29 22:00:48 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P0             58W /  300W |   16127MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [20]:
# Save trained model and tokenizer
trainer.save_model()
tokenizer.save_pretrained(output_dir)

('trained_weigths/tokenizer_config.json',
 'trained_weigths/special_tokens_map.json',
 'trained_weigths/tokenizer.json')

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/runs

In [19]:
y_pred = predict(test, model, tokenizer)
evaluate(y_true, y_pred)

  0%|          | 1/400 [00:00<01:23,  4.78it/s]

[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: It is truly a beautiful story. What do you think is the overall purpose of this story? ANSWER: The overall purpose of the story about La Llorona is to convey a message about the significance of maternal love, the impact of loss, and the emotional bond that exists between a mother and a child. This love in this story transcends even the worst tragedy, so much so that it causes La Llorona to continue mourning the passing of her children even in the afterlife.] = selected'}]
selected
selected


  1%|          | 3/400 [00:00<01:19,  5.01it/s]

[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: How do you think Rosa\'s courage changed the way the villagers saw La Llorona? Does it show us that when we try to understand and feel for others, our fears and the stories we believe about our culture can actually change? ANSWER: Rosa\'s encounter with La Llorona is a significant example of how understanding can transform fear into empathy. The shift indicates that our deepest fears often originate from the unknown or misunderstood aspects of the cultural narratives we have grown up with. By confronting these fears with an open heart, as Rosa did, we allow empathy to guide us toward a deeper understanding. This not only changes our personal perceptions but can also affect communities, reshaping collective beliefs and traditions. It is a testament to the power of individual experiences to influence and

  1%|▏         | 5/400 [00:01<01:34,  4.18it/s]

...

[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: So her tales is directed for kids?  ANSWER: Yes, her tale is adapted and written intended to a young audience, as it was a night time bed story] = rejected'}]
rejected
rejected
[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: What does the disappearing of La Llorona into the mist at the end signify? ANSWER: La Llorona disappearing into the mist is like her spirit finally finding peace. By meeting Rosa and seeing her promise to love her family always, La Llorona changes and goes away, maybe because she finally found what she had been looking for for so long.] = selected'}]
selected
selected


100%|█████████▉| 398/400 [01:15<00:00,  5.43it/s]

[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: What country does the La Llorona myth come from? ANSWER: La Llorona stems from Mexico and other Latin American countries] = rejected'}]
rejected
rejected
[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION:  Can you tell me about the horror film of La Llorona?  ANSWER:  Sure!  There are multiple films telling the story of La Lorona including three main ones, then a number of low-budget movies and other movies loosely based off the story.  The first of the main films was the 1933â€™s La Llorona which was filmed in Mexico.  Next came the 1960â€™s film by Rene Cardona also titled La Llorona and filmed in Mexico.  Lastly, the third film came out in 1963 and was titled The Cu

100%|██████████| 400/400 [01:15<00:00,  5.30it/s]

[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: Why would that be a reasonable prediction? Wouldn\'t the story unfold just as it did, regardless of the age of the person who encounters La Llorona? ANSWER: It\'s possible the story could unfold similarly with a different character. Still, it is reasonable to predict that a different narrative would follow from differently aged characters. Earnestness and genuine curiosity are often associated with children, such as Rosa, and could explain why children have the potential to bring out of others, including adults like the townspeople, greater openness, vulnerability, and a more gentle approach. In this story, the child\'s earnest appeal to La Llorona, whom she has been taught to fear, may have reminded La Llorona of her own lost children, prompting La Llorona to share with Rosa heartfelt advice.] = selec

In [21]:
import gc

del [model, tokenizer, peft_config, trainer, train_data, eval_data, bnb_config, training_arguments]
del [df, X_train, X_eval]
del [TrainingArguments, SFTTrainer, LoraConfig, BitsAndBytesConfig]

In [22]:
for _ in range(100):
    torch.cuda.empty_cache()
    gc.collect()

In [23]:
from peft import AutoPeftModelForCausalLM

finetuned_model = "./trained_weigths/"
compute_dtype = getattr(torch, "float16")
tokenizer = AutoTokenizer.from_pretrained("/content/llama3-8b-chat-hf")

model = AutoPeftModelForCausalLM.from_pretrained(
     finetuned_model,
     torch_dtype=compute_dtype,
     return_dict=False,
     low_cpu_mem_usage=True,
     device_map="auto",
)

merged_model = model.merge_and_unload()
merged_model.save_pretrained("./merged_model",safe_serialization=True, max_shard_size="2GB")
tokenizer.save_pretrained("./merged_model")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('./merged_model/tokenizer_config.json',
 './merged_model/special_tokens_map.json',
 './merged_model/tokenizer.json')

In [24]:
!ls

llama3-8b-chat-hf  merged_model  merged_raw_data.csv  trained_weigths


In [25]:
y_pred = predict(test, merged_model, tokenizer)
evaluate(y_true, y_pred)

  0%|          | 2/400 [00:00<00:49,  7.99it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: It is truly a beautiful story. What do you think is the overall purpose of this story? ANSWER: The overall purpose of the story about La Llorona is to convey a message about the significance of maternal love, the impact of loss, and the emotional bond that exists between a mother and a child. This love in this story transcends even the worst tragedy, so much so that it causes La Llorona to continue mourning the passing of her children even in the afterlife.] = selected'}]
selected
selected
[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: How do you think Rosa\'s courage changed the way the villagers saw La Llorona? Does it show us that when we try to understand and f

  1%|          | 4/400 [00:00<00:54,  7.26it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  2%|▏         | 6/400 [00:00<00:45,  8.72it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: I just read this story about "La Llorona", which I think is beautiful, is this the original story?  "In a quiet village by a silver river, everyone spoke in hushed tones about La Llorona, the Weeping Woman. She was once a beautiful maiden named Maria, with hair as dark as night and eyes that shone like twinkling stars.  Maria loved two things more than anything in the world: her two children and the glistening river that flowed near her home. Every evening, she\'d take her children to play by its banks, their laughter echoing in the air.  One tragic day, while playing near the river, her children slipped away into its currents. Heartbroken, Maria searched day and night but could not find them. Overwhelmed by grief, she would wander the riverbanks, her wails of sorrow becoming a haunting lullaby.  As ye

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  2%|▏         | 8/400 [00:00<00:41,  9.45it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: So this story is kind of the opposite of a ghost story?  ANSWER: In this story, La Llorona is depicted as a supernatural being, or ghost.  However, typically a ghost story serves the purpose of scaring the listener.  This story depicts La Llorona, or the "ghost" as a grieving, but loving character who holds no ill intentions.  In that respect, yes, it could be viewed as the opposite of a ghost story.] = selected'}]
selected
selected
[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: User:Why did the villagers remembered La Llorona? ANSWER: The villagers were afraid of La Llorona, because after loosing her children by the river, she would wander the riverbanks, her wail

 99%|█████████▉| 396/400 [00:37<00:00, 10.79it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
100%|█████████▉| 398/400 [00:37<00:00, 10.76it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: What does the disappearing of La Llorona into the mist at the end signify? ANSWER: La Llorona disappearing into the mist is like her spirit finally finding peace. By meeting Rosa and seeing her promise to love her family always, La Llorona changes and goes away, maybe because she finally found what she had been looking for for so long.] = selected'}]
selected
selected
[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: What country does the La Llorona myth come from? ANSWER: La Llorona stems from Mexico and other Latin American countries] = rejected'}]
rejected
rejected
[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to i

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
100%|██████████| 400/400 [00:37<00:00, 10.68it/s]


[{'generated_text': 'Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected"\n\n     [ QUESTION: Why would that be a reasonable prediction? Wouldn\'t the story unfold just as it did, regardless of the age of the person who encounters La Llorona? ANSWER: It\'s possible the story could unfold similarly with a different character. Still, it is reasonable to predict that a different narrative would follow from differently aged characters. Earnestness and genuine curiosity are often associated with children, such as Rosa, and could explain why children have the potential to bring out of others, including adults like the townspeople, greater openness, vulnerability, and a more gentle approach. In this story, the child\'s earnest appeal to La Llorona, whom she has been taught to fear, may have reminded La Llorona of her own lost children, prompting La Llorona to share with Rosa heartfelt advice.] = selec

In [30]:
!gsutil -m cp -r /content/merged_model gs://llm-finetune/peft/base_model/

Copying file:///content/merged_model/model.safetensors.index.json [Content-Type=application/json]...
Copying file:///content/merged_model/model-00009-of-00009.safetensors [Content-Type=application/octet-stream]...
Copying file:///content/merged_model/model-00001-of-00009.safetensors [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects

In [40]:
!mv merged_model invisi-hire

In [41]:
!ls

invisi-hire  llama3-8b-chat-hf	merged_raw_data.csv  trained_weigths


In [42]:
!gsutil -m cp -r /content/invisi-hire gs://llm-finetune/peft/base_model/

Copying file:///content/invisi-hire/model.safetensors.index.json [Content-Type=application/json]...
Copying file:///content/invisi-hire/model-00009-of-00009.safetensors [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file:///content/invisi-hire/model-00001-of-00009.safetensors [Content-Type=application/octet-stream]...


In [43]:

# Import the necessary packages

import json
import os
from datetime import datetime
from typing import Tuple

from google.cloud import aiplatform

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. [Optional] [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the specified region (`REGION`). Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, please change the value yourself below.
now = datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_URI = "gs://llm-hosting-mixtral"  # @param {type:"string"}

if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}"
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )
print(f"Using this GCS Bucket: {BUCKET_URI}")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(BUCKET_URI, "llama3")


# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Gets the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)


# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID





# The pre-built serving docker images.
VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20240508_0916_RC02"


def get_job_name_with_datetime(prefix: str) -> str:
    """Gets the job name with date time when triggering deployment jobs in
    Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def deploy_model_vllm(
    model_name: str,
    model_id: str,
    service_account: str,
    machine_type: str = "g2-standard-8",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    gpu_memory_utilization: float = 0.9,
    max_model_len: int = 4096,
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys trained models with vLLM into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")

    vllm_args = [
        "--host=0.0.0.0",
        "--port=7080",
        f"--model={model_id}",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        f"--gpu-memory-utilization={gpu_memory_utilization}",
        f"--max-model-len={max_model_len}",
        "--disable-log-stats",
    ]

    env_vars = {"MODEL_ID": model_id, "DEPLOY_SOURCE": "notebook"}
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=VLLM_DOCKER_URI,
        serving_container_command=["python", "-m", "vllm.entrypoints.api_server"],
        serving_container_args=vllm_args,
        serving_container_ports=[7080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/ping",
        serving_container_environment_variables=env_vars,
    )
    print(
        f"Deploying {model_name} on {machine_type} with {accelerator_count} {accelerator_type} GPU(s)."
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    print("endpoint_name:", endpoint.name)

    return model, endpoint


SERVICE_ENDPOINT = "aiplatform.googleapis.com"


def get_quota(project_id: str, region: str, resource_id: str) -> int:
    """Returns the quota for a resource in a region. Returns -1 if can not figure out the quota."""
    quota_list_output = !gcloud alpha services quota list --service=$SERVICE_ENDPOINT  --consumer=projects/$project_id --filter="$SERVICE_ENDPOINT/$resource_id" --format=json
    # Use '.s' on the command output because it is an SList type.
    quota_data = json.loads(quota_list_output.s)
    if len(quota_data) == 0 or "consumerQuotaLimits" not in quota_data[0]:
        return -1
    if (
        len(quota_data[0]["consumerQuotaLimits"]) == 0
        or "quotaBuckets" not in quota_data[0]["consumerQuotaLimits"][0]
    ):
        return -1
    all_regions_data = quota_data[0]["consumerQuotaLimits"][0]["quotaBuckets"]
    for region_data in all_regions_data:
        if (
            region_data.get("dimensions")
            and region_data["dimensions"]["region"] == region
        ):
            if "effectiveLimit" in region_data:
                return int(region_data["effectiveLimit"])
            else:
                return 0
    return -1


def get_resource_id(accelerator_type: str, is_for_training: bool) -> str:
    """Returns the resource id for a given accelerator type and the use case.
    Args:
      accelerator_type: The accelerator type.
      is_for_training: Whether the resource is used for training. Set false
      for serving use case.
    Returns:
      The resource id.
    """
    training_accelerator_map = {
        "NVIDIA_TESLA_V100": "custom_model_training_nvidia_v100_gpus",
        "NVIDIA_L4": "custom_model_training_nvidia_l4_gpus",
        "NVIDIA_TESLA_A100": "custom_model_training_nvidia_a100_gpus",
    }
    serving_accelerator_map = {
        "NVIDIA_TESLA_V100": "custom_model_serving_nvidia_v100_gpus",
        "NVIDIA_L4": "custom_model_serving_nvidia_l4_gpus",
        "NVIDIA_TESLA_A100": "custom_model_serving_nvidia_a100_gpus",
    }
    if is_for_training:
        if accelerator_type in training_accelerator_map:
            return training_accelerator_map[accelerator_type]
        else:
            raise ValueError(
                f"Could not find accelerator type: {accelerator_type} for training."
            )
    else:
        if accelerator_type in serving_accelerator_map:
            return serving_accelerator_map[accelerator_type]
        else:
            raise ValueError(
                f"Could not find accelerator type: {accelerator_type} for serving."
            )


def check_quota(
    project_id: str,
    region: str,
    accelerator_type: str,
    accelerator_count: int,
    is_for_training: bool,
):
    """Checks if the project and the region has the required quota."""
    resource_id = get_resource_id(accelerator_type, is_for_training)
    quota = get_quota(project_id, region, resource_id)
    quota_request_instruction = (
        "Either use "
        "a different region or request additional quota. Follow "
        "instructions here "
        "https://cloud.google.com/docs/quotas/view-manage#requesting_higher_quota"
        " to check quota in a region or request additional quota for "
        "your project."
    )
    if quota == -1:
        raise ValueError(
            f"""Quota not found for: {resource_id} in {region}.
            {quota_request_instruction}"""
        )
    if quota < accelerator_count:
        raise ValueError(
            f"""Quota not enough for {resource_id} in {region}:
            {quota} < {accelerator_count}.
            {quota_request_instruction}"""
        )

Enabling Vertex AI API and Compute Engine API.
Operation "operations/acat.p2-49759099991-f6acfe66-f57a-413f-85a1-f2865720decf" finished successfully.
Using this GCS Bucket: gs://llm-hosting-mixtral
Initializing Vertex AI API.
Using this default Service Account: 49759099991-compute@developer.gserviceaccount.com
No changes made to gs://llm-hosting-mixtral/
Updated property [core/project].


In [ ]:
base_model_name = "invisi-hire-8b"
model_id = os.path.join(MODEL_BUCKET, base_model_name)

model_id = "gs://llm-finetune/peft/base_model/invisi-hire/"

accelerator_type = "NVIDIA_L4"  # @param ["NVIDIA_L4", "NVIDIA_TESLA_A100"]

if "8b" in base_model_name:
    if accelerator_type == "NVIDIA_L4":
        # L4 serving is more cost efficient than V100 serving.
        machine_type = "g2-standard-8"
        accelerator_count = 1
    elif accelerator_type == "NVIDIA_TESLA_A100":
        machine_type = "a2-highgpu-1g"
        accelerator_count = 1
    else:
        raise ValueError(
            f"Recommended GPU setting not found for: {accelerator_type} and {base_model_name}."
        )
elif "70b" in base_model_name:
    # If you do not have access to 4 A100 (40G) GPUs, you may serve LLaMA3 70B
    # models with 8 L4 (24G) GPUs.
    # Note that with the default timeout threshold of Vertex endpoints, you should
    # set a `max_tokens` configuration of around 1,000 tokens or fewer. If you need
    # longer generated sequences, please file a request with Vertex to allowlist
    # your project for a longer timeout threshold with Vertex endpoints.
    if accelerator_type == "NVIDIA_L4":
        machine_type = "g2-standard-96"
        accelerator_count = 8
    elif accelerator_type == "NVIDIA_TESLA_A100":
        machine_type = "a2-highgpu-4g"
        accelerator_count = 4
    else:
        raise ValueError(
            f"Recommended GPU setting not found for: {accelerator_type} and {base_model_name}."
        )
else:
    raise ValueError(
        f"Recommended GPU setting not found for: {accelerator_type} and {base_model_name}."
    )
check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    is_for_training=False,
)

gpu_memory_utilization = 0.85
max_model_len = 8192  # Maximum context length.

model, endpoint = deploy_model_vllm(
    model_name="invisi-hire-v1",
    model_id=model_id,
    service_account=SERVICE_ACCOUNT,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    gpu_memory_utilization=gpu_memory_utilization,
    max_model_len=max_model_len,
)
# @markdown Click "Show Code" to see more details.

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/49759099991/locations/us-central1/endpoints/4879991044860542976/operations/6356686047736233984
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/49759099991/locations/us-central1/endpoints/4879991044860542976
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/49759099991/locations/us-central1/endpoints/4879991044860542976')
INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/49759099991/locations/us-central1/models/7711131231802359808/operations/8543605879289675776
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/49759099991/locations/us-central1/models/7711131231802359808@1
INFO:google.cloud.aiplatform.models:To use this Model in anothe

Deploying invisi-hire-v1 on g2-standard-8 with 1 NVIDIA_L4 GPU(s).


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/49759099991/locations/us-central1/endpoints/4879991044860542976/operations/7011959793518641152


In [45]:
X_test.head()

,Response
3978,Analyze the the of Question and answer enclose...
3357,Analyze the the of Question and answer enclose...
671,Analyze the the of Question and answer enclose...
1133,Analyze the the of Question and answer enclose...
2112,Analyze the the of Question and answer enclose...


In [49]:
# @title Predict

# @markdown Once deployment succeeds, you can send requests to the endpoint with text prompts. Sampling parameters supported by vLLM can be found [here](https://github.com/vllm-project/vllm/blob/2e8e49fce3775e7704d413b2f02da6d7c99525c9/vllm/sampling_params.py#L23-L64).

# @markdown Example:

# @markdown ```
# @markdown Human: What is a car?
# @markdown Assistant:  A car, or a motor car, is a road-connected human-transportation system used to move people or goods from one place to another. The term also encompasses a wide range of vehicles, including motorboats, trains, and aircrafts. Cars typically have four wheels, a cabin for passengers, and an engine or motor. They have been around since the early 19th century and are now one of the most popular forms of transportation, used for daily commuting, shopping, and other purposes.
# @markdown ```
# @markdown Additionally, you can moderate the generated text with Vertex AI. See [Moderate text documentation](https://cloud.google.com/natural-language/docs/moderating-text) for more details.

prompt = "Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label \"selected\" or \"rejected\" [ QUESTION: It is truly a beautiful story. What do you think is the overall purpose of this story? ANSWER: The overall purpose of the story about La Llorona is to convey a message about the significance of maternal love, the impact of loss, and the emotional bond that exists between a mother and a child. This love in this story transcends even the worst tragedy, so much so that it causes La Llorona to continue mourning the passing of her children even in the afterlife.] ="  # @param {type: "string"}
max_tokens = 1  # @param {type:"integer"}
temperature = 0  # @param {type:"number"}
top_p = 1.0  # @param {type:"number"}
top_k = 1  # @param {type:"integer"}
raw_response = False  # @param {type:"boolean"}

# Overides parameters for inferences.
# If you encounter the issue like `ServiceUnavailable: 503 Took too long to respond when processing`,
# you can reduce the maximum number of output tokens, such as set max_tokens as 20.
instances = [
    {
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "raw_response": raw_response,
    },
]
response = endpoint.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

# @markdown Click "Show Code" to see more details.

Prompt:
Analyze the the of Question and answer enclosed in square brackets. leading to informed hiring decisions. Classify label "selected" or "rejected" [ QUESTION: It is truly a beautiful story. What do you think is the overall purpose of this story? ANSWER: The overall purpose of the story about La Llorona is to convey a message about the significance of maternal love, the impact of loss, and the emotional bond that exists between a mother and a child. This love in this story transcends even the worst tragedy, so much so that it causes La Llorona to continue mourning the passing of her children even in the afterlife.] =
Output:
 selected
